<a href="https://colab.research.google.com/github/PurvanshBarodia/price_predictor/blob/main/Purv_House_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What we are going to do?** (problem statment)

We wil try to train the model to predict house price and then see how much better a model can pridict on unseen data using linear regression.

Basic Idea of Linear Regression.

-->

-->

Which dataSet we will use?

California Housing Prices

https://www.kaggle.com/camnugent/california-housing-prices?select=housing.csv


In [1]:
# Upload downloaded csv file
from google.colab import files
uploaded = files.upload()

Saving housing.csv to housing.csv


In [32]:
# What is pandas and how it is useful?
import pandas as pd

In [33]:
# What is numpy? What benifits it gives?
import numpy as np

In [34]:
df = pd.read_csv('housing.csv')

Let's explore the dataset.

In [35]:
# Dataset contain 20640 rows and 10 cols.
df.shape 

(20640, 10)

In [36]:
# See first five rows of dataset.
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [37]:
# Let's understand Which are the columns and what it represents?
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

There are 10 cols:

1. longitude: A measure of how far west a house is; a higher value is farther west

2. latitude: A measure of how far north a house is; a higher value is farther north

3. housingMedianAge: Median age of a house within a block; a lower number is a newer building

4. totalRooms: Total number of rooms within a block

5. totalBedrooms: Total number of bedrooms within a block

6. population: Total number of people residing within a block

7. households: Total number of households, a group of people residing within a home unit, for a block

8. medianIncome: Median income for households within a block of houses (measured in tens of thousands of US Dollars)

9. medianHouseValue: Median house value for households within a block (measured in US Dollars)

10. oceanProximity: Location of the house w.r.t ocean/sea

 Above data description tells that rather than looking for a single house we are going to look at all houses whithin a block to predict house price.

In [38]:
# Let's take statistical overview of each numarical cols
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [39]:
# optional
# what standard deviation suggest? high std means what?

In [40]:
# Now, rename the column names as some col names are....
new_columns = ['longitude','latitude','house_age','rooms','bedrooms','poputation','households','incom','price','ocen_proximity']
df.columns = new_columns

In [41]:
# Let's apply data cleaning, so that we can use our data to train the model

In [42]:
# Check howmany null values are there in each columns.

In [43]:
df.isnull().sum()

longitude           0
latitude            0
house_age           0
rooms               0
bedrooms          207
poputation          0
households          0
incom               0
price               0
ocen_proximity      0
dtype: int64

Here we have 207 (0.01%) missing values for bedrooms, we can:
1. Drop the entire row wich have missing value.

2. We can fill those null vlaues with appropriate statistic (mean, median or mode)

In [44]:
# First drop the entire row which have missing value.
df.dropna(subset= ['bedrooms'],inplace=False)

,longitude,latitude,house_age,rooms,bedrooms,poputation,households,incom,price,ocen_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [45]:
# It had droped 207 rows in which value of bedrooms was null.
# But it is not a good idea to drop values as we will loose informations.
# So let's replace it by mean value of bedrooms

In [46]:
df['bedrooms'].fillna(value=np.mean(df['bedrooms']),inplace=True)

In [47]:
df.isnull().sum()

longitude         0
latitude          0
house_age         0
rooms             0
bedrooms          0
poputation        0
households        0
incom             0
price             0
ocen_proximity    0
dtype: int64

In [48]:
df.dtypes

longitude         float64
latitude          float64
house_age         float64
rooms             float64
bedrooms          float64
poputation        float64
households        float64
incom             float64
price             float64
ocen_proximity     object
dtype: object

In [49]:
# Let's convert categorical feture into numerical feture.
df['ocen_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

We will asign numeric value as follow:

0  --> NEAR BAY

1  --> <1H OCEAN

2  --> INLAND

3  --> Near OCEAN

4  --> ISLAND


In [52]:
df['ocen_proximity'] = df.ocen_proximity.str.replace('NEAR BAY','0')
df['ocen_proximity'] = df.ocen_proximity.str.replace('<1H OCEAN','1')
df['ocen_proximity'] = df.ocen_proximity.str.replace('INLAND','2')
df['ocen_proximity'] = df.ocen_proximity.str.replace('NEAR OCEAN','3')
df['ocen_proximity'] = df.ocen_proximity.str.replace('ISLAND','4')

In [53]:
df['ocen_proximity'] = df.ocen_proximity.astype('float64')

In [54]:
df.head()

,longitude,latitude,house_age,rooms,bedrooms,poputation,households,incom,price,ocen_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0


Now ocen_proximity (categorical value) converted into Numerical feature.

**Does our dataset follows assumption of Linear Regression?**

In [ ]:
# Let's check...